# IE - Iteration 3 Open Data Analysis
## Training models for DES provider's caseload projection

Team by MP32 Bright.

Author: Supawee Rattanajitdamrong

Environment: Python 3.7.6

**How to use**: To make prediction, run the codes under
1) [Data Preparation](#prep) and
2) [Make prediction](#pred) sections. Change the input value to your preferred period. 


**Note: Before running this file, make sure you have run `'des_performance.py'` to update the 'DES_PERFORMANCE_NUMERICAL.csv'.**



### Import library

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, KFold

## Data Preparation<a class="anchor" id="prep"></a>

### Import Data

In [2]:
df = pd.read_csv('../Dataset/DES_PERFORMANCE_NUMERICAL.csv')

In [3]:
data = df[['Year','Month','Total']]
data

Year      Month   Total
0    2011       July  145867
1    2011     August  146282
2    2011  September  147229
3    2011    October  147583
4    2011   November  148594
..    ...        ...     ...
105  2020      April  268773
106  2020        May  267661
107  2020       June  283981
108  2020       July  290252
109  2020     August  291119

[110 rows x 3 columns]

In [4]:
# change string month to number
to_num = {'January':1, 'February':2, 'March':3,'April':4,'May':5,'June':6,'July':7,'August':8,'September':9,\
             'October':10,'November':11,'December':12}

data.replace({'Month':to_num},inplace=True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6666: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regex=regex,


### Create training and test set

In [5]:
# randomly split data and set seed (90-10)
train_set=data.sample(frac=0.9,random_state=10) 
test_set=data.drop(train_set.index)

train_Y = train_set['Total'] 
test_Y = test_set['Total']
train_X = train_set[['Year','Month']]
test_X = test_set[['Year','Month']]

## Make prediction<a class="anchor" id="pred"></a>

Predict number of caseload for September 2020.

In [6]:
# change the input here
year = 2020
month = 9
input_X = np.array([[year, month]])

In [7]:
# poly
LMmodel = LinearRegression()
PLmodel = PolynomialFeatures(3)
transform_train_X = PLmodel.fit_transform(train_X)

LMmodel.fit(transform_train_X, train_Y)

transform_train_X = PLmodel.fit_transform(input_X)
prediction = LMmodel.predict(transform_train_X)
print("Prediction for "+str(month)+", "+ str(year)+":  {:,.2f}".format(round(prediction[0])))

Prediction for 9, 2020:  299,950.00


The result will need to be update in 'desperformance.html' for both normal and color blind versions. 





------------------------------------------------

# Model Selection and Evaluation

## Model 1 Linear Regressions

The implementation will be shown in model evaluation section since there is no tuning required.

## Model 2 Polynomial Regressions

#### Model Selection
Cross validation to find the optimal degree for polynomial regression. Using training and validation set. The cross validation MSE are shown below.

In [8]:
MSE = []
svrMSE = []
PolyMSE = {}
LMmodel = LinearRegression()

rkf = KFold(n_splits=5, random_state=100, shuffle=True )

for i in range(1,10):
    MSE = []
    for train, val in rkf.split(train_set):

        train_X = train_set.iloc[train,:][['Year','Month']]
        train_Y = train_set.iloc[train,:][['Total']]

        val_X = train_set.iloc[val,:][['Year','Month']]
        val_Y = train_set.iloc[val,:][['Total']]
        
        PLmodel = PolynomialFeatures(int(i))
        transform_train_X = PLmodel.fit_transform(train_X)

        LMmodel.fit(transform_train_X, train_Y)
        transform_val_X = PLmodel.fit_transform(val_X)
        PLpred = LMmodel.predict(transform_val_X)
        MSE.append(mean_squared_error(PLpred, val_Y))
        
    # multiply with 10**6 to reduce the scale
    PolyMSE[i] = round(sum(MSE)/(len(MSE)*10**6),2)

print(PolyMSE)
print('\n')
print("Poly Degree 3")
print(PolyMSE[3])

{1: 279.12, 2: 90.82, 3: 53.26, 4: 55.43, 5: 56.0, 6: 58.17, 7: 58.38, 8: 58.33, 9: 59.14}


Poly Degree 3
53.26


#### The best model is 3-degree polynomial with the lowest cross validation MSE.

## Model 3 : Support Vector Regression (SVR)
#### Model Selection
Cross Validation - Hyperparameter tunning for SVR

In [9]:
param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf', 'sigmoid']}
grid = GridSearchCV(SVR(),param_grid,refit=True,verbose=2)
grid.fit(train_X,train_Y)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] ....................... C=0.1, gamma=1, kernel=rbf, total=   0.0s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] ....................... C=0.1, gamma=1, kernel=rbf, total=   0.0s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] ....................... C=0.1, gamma=1, kernel=rbf, total=   0.0s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] ....................... C=0.1, gamma=1, kernel=rbf, total=   0.0s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] ....................... C=0.1, gamma=1, kernel=rbf, total=   0.0s
[CV] C=0.1, gamma=1, kernel=sigmoid ..................................
[CV] ................... C=0.1, gamma=1, kernel=sigmoid, total=   0.0s
[CV] C=0.1, gamma=1, kernel=sigmoid ..................................
[CV] ..........

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/s

[CV] ................. C=0.1, gamma=0.1, kernel=sigmoid, total=   0.0s
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV] .................... C=0.1, gamma=0.01, kernel=rbf, total=   0.0s
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV] .................... C=0.1, gamma=0.01, kernel=rbf, total=   0.0s
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV] .................... C=0.1, gamma=0.01, kernel=rbf, total=   0.0s
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV] .................... C=0.1, gamma=0.01, kernel=rbf, total=   0.0s
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV] .................... C=0.1, gamma=0.01, kernel=rbf, total=   0.0s
[CV] C=0.1, gamma=0.01, kernel=sigmoid ...............................
[CV] ................ C=0.1, gamma=0.01, kernel=sigmoid, total=   0.0s
[CV] C=0.1, gamma=0.01, kernel=sigmoid ...............................
[CV] .

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

[CV] ......................... C=1, gamma=1, kernel=rbf, total=   0.0s
[CV] C=1, gamma=1, kernel=sigmoid ....................................
[CV] ..................... C=1, gamma=1, kernel=sigmoid, total=   0.0s
[CV] C=1, gamma=1, kernel=sigmoid ....................................
[CV] ..................... C=1, gamma=1, kernel=sigmoid, total=   0.0s
[CV] C=1, gamma=1, kernel=sigmoid ....................................
[CV] ..................... C=1, gamma=1, kernel=sigmoid, total=   0.0s
[CV] C=1, gamma=1, kernel=sigmoid ....................................
[CV] ..................... C=1, gamma=1, kernel=sigmoid, total=   0.0s
[CV] C=1, gamma=1, kernel=sigmoid ....................................
[CV] ..................... C=1, gamma=1, kernel=sigmoid, total=   0.0s
[CV] C=1, gamma=0.1, kernel=rbf ......................................
[CV] ....................... C=1, gamma=0.1, kernel=rbf, total=   0.0s
[CV] C=1, gamma=0.1, kernel=rbf ......................................
[CV] .

...

[CV] ................. C=10, gamma=0.01, kernel=sigmoid, total=   0.0s
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV] .................... C=10, gamma=0.001, kernel=rbf, total=   0.0s
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV] .................... C=10, gamma=0.001, kernel=rbf, total=   0.0s
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV] .................... C=10, gamma=0.001, kernel=rbf, total=   0.0s
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV] .................... C=10, gamma=0.001, kernel=rbf, total=   0.0s
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV] .................... C=10, gamma=0.001, kernel=rbf, total=   0.0s
[CV] C=10, gamma=0.001, kernel=sigmoid ...............................
[CV] ................ C=10, gamma=0.001, kernel=sigmoid, total=   0.0s
[CV] C=10, gamma=0.001, kernel=sigmoid ...............................
[CV] .

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

[CV] ................. C=100, gamma=0.1, kernel=sigmoid, total=   0.0s
[CV] C=100, gamma=0.01, kernel=rbf ...................................
[CV] .................... C=100, gamma=0.01, kernel=rbf, total=   0.0s
[CV] C=100, gamma=0.01, kernel=rbf ...................................
[CV] .................... C=100, gamma=0.01, kernel=rbf, total=   0.0s
[CV] C=100, gamma=0.01, kernel=rbf ...................................
[CV] .................... C=100, gamma=0.01, kernel=rbf, total=   0.0s
[CV] C=100, gamma=0.01, kernel=rbf ...................................
[CV] .................... C=100, gamma=0.01, kernel=rbf, total=   0.0s
[CV] C=100, gamma=0.01, kernel=rbf ...................................
[CV] .................... C=100, gamma=0.01, kernel=rbf, total=   0.0s
[CV] C=100, gamma=0.01, kernel=sigmoid ...............................
[CV] ................ C=100, gamma=0.01, kernel=sigmoid, total=   0.0s
[CV] C=100, gamma=0.01, kernel=sigmoid ...............................
[CV] .

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

GridSearchCV(cv=None, error_score=nan,
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='scale', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001],
                         'kernel': ['rbf', 'sigmoid']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=2)

In [10]:
# best parametor
print(grid.best_estimator_)

SVR(C=100, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)


## Model Evaluation
Find the best model among the three by comparing the test MSEs.

In [11]:
# linear 
LMmodel = LinearRegression()
LMmodel.fit(train_X, train_Y )
LMpred = LMmodel.predict(test_X)
LM_TestMSE = mean_squared_error(LMpred, test_Y)
round(LM_TestMSE/10**6,2)

266.55

In [12]:
# poly
PLmodel = PolynomialFeatures(3)
transform_train_X = PLmodel.fit_transform(train_X)

LMmodel.fit(transform_train_X, train_Y)
transform_test_X = PLmodel.fit_transform(test_X)
PLpred = LMmodel.predict(transform_test_X)
PL_TestMSE = mean_squared_error(PLpred, test_Y)
round(PL_TestMSE/10**6,2)

62.89

In [13]:
# SVR
SVRmodel = SVR(C=100, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1, kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
SVRmodel.fit(train_X,train_Y)
SVRpred = SVRmodel.predict(test_X)
SVR_TestMSE = mean_squared_error(SVRpred, test_Y)
round(SVR_TestMSE/10**6,2)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


1519.32

## The Best Model

#### The best model is 3-degree polynomial.